# Working with `weather.gov` API

MCS 275 Spring 2024 - Emily Dumas

This notebook was the work-in-progress for `weather.py`.

In [1]:
from urllib.request import urlopen
import time
import json

In [3]:
# Figure out which weather.gov "grid" contains Chicago
# Google search says Chicago lat/long are 41.8781 N and 87.6298 W
time.sleep(1) # rate limiter!
with urlopen("https://api.weather.gov/points/41.8781,-87.6298") as fp:
    latlong_data = json.load(fp)

In [4]:
latlong_data

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   's': 'https://schema.org/',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#',
   'geometry': {'@id': 's:GeoCoordinates', '@type': 'geo:wktLiteral'},
   'city': 's:addressLocality',
   'state': 's:addressRegion',
   'distance': {'@id': 's:Distance', '@type': 's:QuantitativeValue'},
   'bearing': {'@type': 's:QuantitativeValue'},
   'value': {'@id': 's:value'},
   'unitCode': {'@id': 's:unitCode', '@type': '@id'},
   'forecastOffice': {'@type': '@id'},
   'forecastGridData': {'@type': '@id'},
   'publicZone': {'@type': '@id'},
   'county': {'@type': '@id'}}],
 'id': 'https://api.weather.gov/points/41.8781,-87.6298',
 'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-87.6298, 41.8781]},
 'properties': {'@id': 'https://api.weather.gov/p

In [5]:
# Dig into the complex return value to find the part where the tell us
# the URL of Chicago's forecast
chicago_forecast_url = latlong_data["properties"]["forecast"]

In [6]:
print(chicago_forecast_url)

https://api.weather.gov/gridpoints/LOT/76,73/forecast


In [7]:
# Grab Chicago's forecast
time.sleep(1)
with urlopen(chicago_forecast_url) as fp:
    chicago_forecast_data = json.load(fp)

In [8]:
chicago_forecast_data

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#'}],
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-87.6307675, 41.8882767],
    [-87.6323584, 41.8665035],
    [-87.603123, 41.8653168],
    [-87.60152579999999, 41.8870899],
    [-87.6307675, 41.8882767]]]},
 'properties': {'updated': '2024-04-10T19:26:44+00:00',
  'units': 'us',
  'forecastGenerator': 'BaselineForecastGenerator',
  'generatedAt': '2024-04-10T19:45:59+00:00',
  'updateTime': '2024-04-10T19:26:44+00:00',
  'validTimes': '2024-04-10T13:00:00+00:00/P7DT12H',
  'elevation': {'unitCode': 'wmoUnit:m', 'value': 180.1368},
  'periods': [{'number': 1,
    'name': 'This Afternoon',
    'startTime': '2024-04-10T14:00:00-05:00',
    'endTime': '2024-04-10T18:00:00-05:00',


In [12]:
# properties -> periods -> list of forecasts -> first element is current one (not e.g. tonight, tomorrow, ...)
current_forecast = chicago_forecast_data["properties"]["periods"][0]

In [13]:
current_forecast

{'number': 1,
 'name': 'This Afternoon',
 'startTime': '2024-04-10T14:00:00-05:00',
 'endTime': '2024-04-10T18:00:00-05:00',
 'isDaytime': True,
 'temperature': 59,
 'temperatureUnit': 'F',
 'temperatureTrend': 'falling',
 'probabilityOfPrecipitation': {'unitCode': 'wmoUnit:percent', 'value': None},
 'dewpoint': {'unitCode': 'wmoUnit:degC', 'value': 3.3333333333333335},
 'relativeHumidity': {'unitCode': 'wmoUnit:percent', 'value': 63},
 'windSpeed': '10 mph',
 'windDirection': 'ENE',
 'icon': 'https://api.weather.gov/icons/land/day/bkn?size=medium',
 'shortForecast': 'Mostly Cloudy',
 'detailedForecast': 'Mostly cloudy. High near 59, with temperatures falling to around 50 in the afternoon. East northeast wind around 10 mph.'}

In [14]:
print("Current Chicago forecast: {}".format(current_forecast["shortForecast"]))
print()
print(current_forecast["detailedForecast"])

Current Chicago forecast: Mostly Cloudy

Mostly cloudy. High near 59, with temperatures falling to around 50 in the afternoon. East northeast wind around 10 mph.
